In [17]:
# -*- coding: Paper1 -*-
"""
This code belongs to the paper:
-- An Iterative Gradient Descent-Based Reinforcement Learning Policy 
   for Active Control of Structural Vibrations, Computers & Structures.
-- Jagajyoti Panda, Mudit Chopra, Vasant Matsagar, Souvik Chakraborty, IIT Delhi.
   
This code is to present RL-based control algorithm in proportional-integral (PI) to state-output feedback.
-- Case study I: Quarter car model 
   State-space representation, Policy parameter update, Efficacy of trained RL-controller
"""
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import os
from numpy import abs as Abs
from numpy import sign as sign
from numpy import sqrt,pi,array
from scipy import signal
 
 
import matplotlib
import matplotlib.pyplot as plt

from control.matlab import *  # MATLAB-like functions
import control as ct
 
import tensorflow as tf
from gym import Env
from gym.spaces import Discrete, Box
 
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# Quarter car model
M = 600  # Body mass (in Kg)
m = 75  # Mass of tyre (in Kg)
Ks = 30  # Spring constant
Cs = 400  # Damping coefficient
Kt = 300  # Spring constant of wheel
n = 2  # DOF
r = 1  # No of Actuator

# MR Damper
V = 20
K1 = 542.18
K2 = 550.16
C1 = 999.4
C2 = 3387.5
alpha  = 3846.2 
beta = 126.84
gamma = 303.83 
A1 = 96.067
N = 3

# State space representation (states - y1, y2, dot(y1), dot(y2)) 
# Continous time
A =np.array([[0 ,0 ,1 ,0 ],
              [0 ,0 ,0 ,1 ],
              [-Ks/M, Ks/M, 0, 0],
              [Ks/m, -1*(Ks+Kt)/m, 0, 0]])
print(A)
# Input matrix (Control force and Earthquake input)
B  = np.array([[0,0,1/M,-1/m]]).T
print(B)
# Influence vector (Earthquake input)
E = np.array([[0,0,0,Kt/m]]).T 
print(E)
# Output matrix (Disp + Accel)
Cd = np.array([[1, 1, 0, 0], [-Ks/M, Ks/M, 0, 0], [Ks/m, -1*(Ks+Kt)/m, 0, 0]])
print(Cd)
Dd = np.array([[0, 1/M, -1/m]]).T
print(Dd)
Ed = np.array([[0, 0, 0]]).T
print(Ed)

In [ ]:
from numpy.linalg import matrix_rank
ko = matrix_rank(ctrb(A, B))
if ko == len(A):
  print("System is controllable")
else:
  print("System is uncontrollable")
del ko

In [ ]:
# Input Road Profile
T = 100
dt = 0.01
t = np.arange(0, T+dt, dt)
omega = 1
amplitude   = 0.1*np.sin(omega*t)
len(t)
plt.figure()
plt.plot(t, amplitude)
plt.title('Road profile')
plt.xlabel('Time(s)')
plt.ylabel('Input(m)')

input = np.array([t,amplitude])
print(input.T.shape)
result=np.savetxt('time.txt', input.T, delimiter=' ')
print(result)

In [ ]:
# Discrete time matrices
Ad = np.identity(2*n)
I = np.identity(2*n)
fact = 1
for x in range(1, 20):
    fact = fact * x
    Ad = Ad + ((A**x)*((dt**x)/fact))
    
Bd = (Ad-I)@np.linalg.inv(A)@B
Ed = (Ad-I)@np.linalg.inv(A)@E
print(Ad, Bd, Ed) 
del I, x

In [ ]:
# System State Space 
# PI controller (continous)
FIc = np.zeros((2*n+r,2*n+r))
FIc[0:2*n,0:2*n] = A
FIc[2*n,0:2*n] = -Cd[0,:]
print(FIc)
gIc = np.zeros((2*n+r,r))
gIc[0:2*n,:] = B
print(gIc)
BIrdc = np.zeros((2*n+r,r))
BIrdc[0:2*n,:] = E
print(BIrdc)
# Discrete
FI = np.zeros((2*n+r,2*n+r))
FI[0:2*n,0:2*n] = Ad
FI[2*n,0:2*n] = Cd[0,:] @ Ad
FI[2*n,2*n] = np.identity(r)
print(FI)
gI = np.zeros((2*n+r,r))
gI[0:2*n,:] = Bd
gI[2*n,:] = Cd[0,:] @ Bd
print(gI)
BIrd = np.zeros((2*n+r,r))
BIrd[0:2*n,:] = Ed
BIrd[2*n,:] = Cd[0,:] @ Ed
print(BIrd)
# Output of PI controller
Cdi = np.zeros((n+r,2*n+r))
Cdi[0:n,0:2*n] = Cd[1:n+1,:]
Cdi[n,2*n] = np.identity(r)
print(Cdi)
Ddi = np.zeros((n+r,r))
Ddi[0:n,:] = Dd[1:n+1,:]
print(Ddi)
ko = matrix_rank(ctrb(FIc, gIc))
if ko == len(FIc):
  print("System is controllable")
else:
  print("System is uncontrollable")
del ko

# Propertional (Continous)
Qp = np.array([[Ks, -Ks, 0, 0],
            [-Ks, Ks+Kt, 0, 0],
            [0, 0, M, 0],
            [0, 0, 0, m]])
(i,j)=gI.shape
R = (10**(0))*np.identity(j)
Kp1, Sp1, ep1 = ct.lqr(A, B, Qp, R)
print(Kp1) # Continous time
xp1 = np.zeros((len(t),2*n))
yp1 = np.zeros((len(t),i))
sys_c = signal.StateSpace(A-B@Kp1, E, Cd-Dd@Kp1, np.array([[0, 0, 0]]).T)
t_c,yp1,xp1 = signal.lsim(sys_c,amplitude,t)

In [ ]:
# LQR design (propertional+Integral) (Case 1)
Qpi = np.zeros((2*n+r,2*n+r))
Qpi[0:2*n,0:2*n] = Qp
out = Cd[0,:] @ Qp
out_arr  = out[np.nonzero(out)]
# Qpi[2*n,2*n] = out_arr
Qpi[2*n,2*n] = Ks
# print(Qpi)
Kd, S, e = ct.dlqr(FI, gI, Qpi, R)
print(Kd)
Kdc, Sc, ec = ct.lqr(FIc, gIc, Qpi, R)
print(Kdc)

# LQR design (propertional+Integral) (Case 2)
Q2 = (Cdi.T) @ np.identity(n+r) @ Cdi
W2 = (Cdi.T) @ np.identity(n+r) @ Ddi
R2 = ((Ddi.T) @ np.identity(n+r) @ Ddi) + R 
Q_bar = Q2 - (W2 @ np.linalg.inv(R2) @ W2.T)
FIc_bar = FIc - (gIc @ np.linalg.inv(R2) @ W2.T)
Kda1c, Sa, ea = ct.lqr(FIc_bar, gIc, Q_bar, R2)
Kdac = Kda1c + (np.linalg.inv(R2) @ W2.T)
print(Kdac)

FI_bar = FI - (gI @ np.linalg.inv(R2) @ W2.T)
Kda1, Sa, ea = ct.dlqr(FI_bar, gI, Q_bar, R2)
Kda = Kda1 + (np.linalg.inv(R2) @ W2.T)
print(Kda)
del S, e, Sa, ea
np.save('Qpi.npy', Qpi)
np.save('ControllerPI_QC.npy', Kdc)

In [ ]:
# Time series analysis (Continous)
xpi = np.zeros((len(t),2*n+r))
xpi1 = np.zeros((len(t),2*n+r))
(i,j) = Cdi.shape
ypi = np.zeros((len(t),i))
ypi1 = np.zeros((len(t),i))
del i,j

sys_c = signal.StateSpace(FIc-gIc@Kdc, BIrdc, Cdi-Ddi@Kdc, np.array([[0, 0, 0]]).T)
t_c,ypi,xpi = signal.lsim(sys_c,amplitude,t)

sys_c1 = signal.StateSpace(FIc_bar-gIc@Kdac, BIrdc, Cdi-Ddi@Kdac, np.array([[0, 0, 0]]).T)
t_c,ypi1,xpi1 = signal.lsim(sys_c,amplitude,t)

for i in range(0,2*n):
    plt.figure()
    plt.plot(t, xp1[:,i], 'r', t, xpi[:,i], 'b', t, xpi1[:,i], 'g')
del i 

for i in range(0,len(Cdi)-1):
    plt.figure()
    plt.plot(t, yp1[:,i+1], 'r', t, ypi[:,i], 'b', t, ypi1[:,i], 'g')
del i

mag_pc, phase_pc, om_pc = bode(ss(FIc-gIc@Kdc, BIrdc, np.array([1, 0, 0, 0, 0])-Ddi[2, :]@Kdc, np.array([0]).T), logspace(-2, 2), plot=True)

In [ ]:
# Initial controller gain (proportional)
Qp = np.array([[1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]])
Qpi_i = np.zeros((2*n+r,2*n+r))
Qpi_i[0:2*n,0:2*n] = Qp
Qpi_i[2*n,2*n] = 1
print(Qpi_i)
Kdc, S, e = ct.lqr(FIc, gIc, Qpi_i, R)
Kdac, S, e = ct.lqr(FIc_bar, gIc, Qpi_i, R)
Kd, S, e = ct.dlqr(FI, gI, Qpi_i, R)

print(Kdc, Kdac, Kd)

In [ ]:
# PI controller (continous)
Bj_pi = gIc @ (-1*Kdc)
Al_pi = FIc.T + Bj_pi.T 
P_pi = lyap(Al_pi, -Qpi)
print(Bj_pi)
print(P_pi)
# PI controller (Discrete)
Bj_pi1 = gI @ (-1*Kd)
Al_pi1 = FI.T + Bj_pi1.T 
P_pi1 = dlyap(Al_pi1, -Qpi)

In [295]:
# Dynamical equation (Rk4) (State)
def dydx(A1, B1, E1, U1, h1, x1):
#   print(x1,U1,h1)
  Bu  = U1*B1
#   print(Bu)
  fi = h1*E1
  k1 = dt*(A1@x1 + Bu@x1 + fi)
  k2 = dt*(A1@(x1 + 0.5*k1)+ Bu@(x1 + 0.5*k1) + fi)
  k3 = dt*(A1@(x1 + 0.5*k2)+ Bu@(x1 + 0.5*k2) + fi)
  k4 = dt*(A1@(x1 + k3)+ Bu@(x1 + k3) + fi)
  x2 = x1 + (k1 + 2*k2 + 2*k3 + k4)/6
  return x2

In [296]:
# Dynamical equation (Rk4) (Adjoint State)
def dydx1(A1, B1, E1, U1, h1, x1):
#   print(x1,U1,h1)
  Bu  = U1*B1
#   print(Bu)
  fi = h1*E1
  k1 = -dt*(A1@x1 + Bu@x1 + fi)
  k2 = -dt*(A1@(x1 + 0.5*k1)+ Bu@(x1 + 0.5*k1) + fi)
  k3 = -dt*(A1@(x1 + 0.5*k2)+ Bu@(x1 + 0.5*k2) + fi)
  k4 = -dt*(A1@(x1 + k3)+ Bu@(x1 + k3) + fi)
  x2 = x1 + (k1 + 2*k2 + 2*k3 + k4)/6
  return x2

In [ ]:
# PI-controller (Continous)
# Policy parameter update
Zmax = 1001 # No of episodes
BK_pi = np.zeros([2*n+r,2*n+r,Zmax])
BK_pi[0:2*n+r,0:2*n+r,0] = Bj_pi  # Initial Stiffness
alpha = 30
djdk_pi = np.zeros([2*n+r,2*n+r,Zmax])  # Gradient
x_pi = np.zeros([len(t), 2*n+r, Zmax])  # State
p_pi = np.zeros([len(t), 2*n+r, Zmax])  # Adjoint State
done = False
Jc_pi = np.zeros([len(t), r, Zmax])  # Cost function
Ec_pi = np.zeros([len(t), r, Zmax])  # Structure's energy
val_arr = np.zeros([len(t), r, Zmax]) # x.T*K*x
s_pi = []  # number of tau_s

for id in range(0, Zmax-1):  
  BKo = BK_pi[0:2*n+r,0:2*n+r,id] 
  print(BKo)
  for tkk in range(0,len(t)-1):
    time_current = t[tkk]
    h = amplitude[tkk] 
    xn = np.array([x_pi[tkk,:,id]]).T
#   print(xn)
    xn1 = dydx(FIc, BKo, BIrdc, 1, h, xn)
#     print(xn1)
    x_pi[tkk+1,:,id] = xn1.T
    En1 = 0.5*((xn1.T @ Qpi)@xn1)
    Ec_pi[tkk+1,:,id] = En1
    Jn1 = Jc_pi[tkk,:,id] + dt*(En1)
    Jc_pi[tkk+1,:,id] = Jn1  # Cost functional
    del xn,xn1
  print(Jn1)  

  # Adjoint State  
  s_pi.append([])
  for tkk in range(0,len(t)-1):
    i = len(t)-1-tkk
#     print(i)
    xn = np.array([x_pi[i,:,id]]).T
    val = (xn.T @ BKo) @ xn
#     print(val)
    val_arr[i,:,id] = val
    if(val==0):
      pn = np.array([p_pi[i,:,id]]).T
      s_pi[id].append(i)
      d_ptau1 = (pn.T@BKo)@xn  # (1*n)@(n*n)@(n*1)
#       print(val)
      d_ptau = (d_ptau1[0,0]*((BKo + BKo.T)@xn))      #1*(1*1)*((n*n)@(n*1))  #############
#       d_ptau = (d_ptau1[0,0]*xn)      #1*(1*1)*((n*n)@(n*1))  #############
#       print(d_ptau)
      pnb1 = pn + d_ptau  # (n*1)
      p_pi[i-1,:,id] = pnb1.T
    else:
      if(val[0]<0.000001 and val[0] >-0.000001):
        s_pi[id].append(i)
      pn = np.array([p_pi[i,:,id]]).T
      F = Qpi@xn 
      pnb1 = dydx1(-FIc.T, -BKo.T, F, 1, 1, pn)  # Backward integration
#       print(pnb1)
      p_pi[i-1,:,id] = pnb1.T
#     print(p[i-1,:,id])
    del xn, val, pn, pnb1, i
    
  # Evaluate dj/dk
  sum = djdk_pi[0:2*n+r,0:2*n+r,id]
  print(len(s_pi[id]))
  if(len(s_pi[id])>0):
    for j in s_pi[id]:
        xn = np.array([x_pi[j,:,id]]).T
        pn = np.array([p_pi[j,:,id]]).T
        sum1 = (pn.T@BKo)@xn  # (1*n)@(n*n)@(n*1)
        sum = sum +  ((-1)*(sum1[0,0]*(xn@xn.T)))  # 1*(1*1)*((n*1)@(1*n))  #############
#     print(sum)
    djdk_pi[0:2*n+r,0:2*n+r,id] = sum
  # Update K
    BK_pi[0:2*n+r,0:2*n+r,id+1] = BK_pi[0:2*n+r,0:2*n+r,id]  - alpha*djdk_pi[0:2*n+r,0:2*n+r,id]
  else:
    print(np.linalg.norm(djdk_pi[0:2*n+r,0:2*n+r,id]))
#     break

In [ ]:
# Figure (dj_dk, cost  functional)
z = np.arange(0, Zmax-1, 1)
n_djdk = np.zeros([Zmax-1, r])
J_Tl = np.zeros([Zmax-1, r])
del id
for id in range(0, Zmax-1):
    norm_djdk = np.linalg.norm(djdk_pi[0:2*n+r,0:2*n+r,id])
    n_djdk[id,:] = norm_djdk
    J_T = Jc_pi[len(t)-1,:,id]
    J_Tl[id, :] = J_T
plt.figure()
plt.plot(z, n_djdk)
plt.figure()
plt.plot(z, J_Tl/J_Tl[0,0])

In [ ]:
print(BK_pi[0:2*n+r,0:2*n+r,Zmax-1])
# Save results
np.save('ControlPIRL_QC.npy', BK_pi[0:2*n+r,0:2*n+r,Zmax-1])


# Figure (Structural responses)
Cd2 = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [-Ks/M, Ks/M, 0, 0, 0], [Ks/m, -1*(Ks+Kt)/m, 0, 0, 0], [0, 0, 0, 0, 1]])
yp_z0 = (Cd2-BK_pi[0:2*n+r,0:2*n+r,0]) @ np.array([x_pi[:,:,0].T])
yp_zmax = (Cd2-BK_pi[0:2*n+r,0:2*n+r,Zmax-1]) @ np.array([x_pi[:,:,Zmax-2].T]) 

for i in range(0,len(Cdi)-1):
    plt.figure()
    plt.plot(t, ypi1[:,i],'r',t, ypi[:,i], 'b', t, yp_zmax[:,i+2,:].T,'g')
    plt.title('Abs Accel')
    plt.xlabel('Time(s)')
    plt.ylabel('Accel(m/s2)')
del i     


In [ ]:
# Testing efficacy of the trained Rl-controller
# Input Road Profile
amplitude1   = 0.5*(np.sin(0.2*t) + np.sin(2*t))
len(t)
plt.figure()
plt.plot(t, amplitude1)
plt.title('Road profile')
plt.xlabel('Time(s)')
plt.ylabel('Input(m)')

# Fast fourier transform (FFT)
sr = 1/dt  # sampling frequency
N1 = 2046  # Number of samples
fstep = sr/N1  # freq interval
f = np.linspace(0, (N1-1)*fstep, N1)

# Perform FFT
X = np.fft.fft(amplitude)
X_mag = np.abs(X)/N1

X1 = np.fft.fft(amplitude1)
X1_mag = np.abs(X1)/N1

f_plot = f[0:int(N1/2+1)]
X_mag_plot = 2*X_mag[0:int(N1/2+1)]
X_mag_plot[0] = X_mag_plot[0] / 2 # DC component does not need to multiply by 2

X1_mag_plot = 2*X1_mag[0:int(N1/2+1)]
X1_mag_plot[0] = X1_mag_plot[0] / 2 # DC component does not need to multiply by 2

# Plot 
fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1)
ax1.plot(f_plot, X_mag_plot, '.-')
ax2.plot(f_plot, X1_mag_plot, '.-')
ax2.set_xlabel('Freq (rad/sec)')
ax1.set_ylabel('FFT Amplitude |X(freq)|')
ax2.set_ylabel('FFT Amplitude |X(freq)|')
ax1.semilogx()
ax2.semilogx()
plt.show()

In [ ]:
BKt = np.load('ControlPIRL_QC.npy')
Kdc = np.load('ControllerPI_QC.npy')
xt = np.zeros([len(t), 2*n+r])  # State
yt = np.zeros([len(t), len(Cd2)])  # State
print(BKt)
print(Kdc)
for tkk in range(0,len(t)-1):
    time_current = t[tkk]
    h = amplitude1[tkk] 
    xn = np.array([xt[tkk,:]]).T
#   print(xn)
    xn1 = dydx(FIc, BKt, BIrdc, 1, h, xn)
#     print(xn1)
    xt[tkk+1,:] = xn1.T
    yn1 = (Cd2-BKt) @ xn1
    yt[tkk+1,:] = yn1.T

In [50]:
# Open loop and optimal control
sys_uc = signal.StateSpace(FIc, BIrdc, Cdi, np.array([[0, 0, 0]]).T)
sys_c = signal.StateSpace(FIc-gIc@Kdc, BIrdc, Cdi-Ddi@Kdc, np.array([[0, 0, 0]]).T)
t_c,yuct,xuct = signal.lsim(sys_uc,amplitude1,t)
t_c,yot,xot = signal.lsim(sys_c,amplitude1,t)


for i in range(2*n):
    plt.figure()
    plt.plot(t, xuct[:,i], 'r', t,xot[:,i],'b', t, xt[:,i],'g')
del i 
for i in range(0,len(Cdi)-1):
    plt.figure()
    plt.plot(t, yuct[:,i], 'r', t, yot[:,i],'b', t, yt[:,i+2],'g')
    
# Maximum value of response
print("Peak Dis of str - ", "Uncontrolled:", max(xuct[:,0], key=abs), "Optimal-PI:", max(xot[:,0], key=abs), "PG-PI:", max(xt[:,0], key=abs))
print("Peak Dis of Tyre - ", "Uncontrolled:", max(xuct[:,1], key=abs), "Optimal-PI:", max(xot[:,1], key=abs),"PG-PI:", max(xt[:,1], key=abs))
print("Peak Vel of str - ", "Uncontrolled:", max(xuct[:,2], key=abs), "Optimal-PI:", max(xot[:,2], key=abs), "PG-PI:", max(xt[:,2], key=abs))
print("Peak Vel of Tyre - ", "Uncontrolled:", max(xuct[:,3], key=abs), "Optimal-PI:", max(xot[:,3], key=abs),"PG-PI:", max(xt[:,3], key=abs))
print("Peak Acc of str - ", "Uncontrolled:", max(yuct[:,0], key=abs), "Optimal-PI:", max(yot[:,0], key=abs), "PG-PI:", max(yt[:,2], key=abs))
print("Peak Acc of Tyre - ", "Uncontrolled:", max(yuct[:,1], key=abs), "Optimal-PI:", max(yot[:,1], key=abs),"PG-PI:", max(yt[:,3], key=abs))